Try and do summarization of CFR sections

In [1]:
import pandas as pd
from transformers import pipeline

c:\ProgramData\anaconda3\envs\conda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the dataframe 
title_number = "12"
df = pd.read_parquet(f"../dataframe/{title_number}.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117306 entries, 0 to 117305
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   p_id          117306 non-null  object
 1   text          117306 non-null  object
 2   child_ids     117306 non-null  object
 3   cfr_links     117306 non-null  object
 4   other_links   117306 non-null  object
 5   link_targets  117306 non-null  object
dtypes: object(6)
memory usage: 5.4+ MB


In [25]:
reg_z_id = '1026.1'
reg_z_df = df[df['p_id'].str.startswith(reg_z_id)].drop_duplicates(subset='p_id')
reg_z_text_list = list(reg_z_df['text'])
reg_z_text_list

["(a)Authority.This part, known as Regulation Z, is issued by the Bureau of Consumer Financial Protection to implement the Federal Truth in Lending Act, which is contained in title I of the Consumer Credit Protection Act, as amended (15 U.S.C. 1601et seq.). This part also implements title XII, section 1204 of the Competitive Equality Banking Act of 1987 (Pub. L. 100–86, 101 Stat. 552). Furthermore, this part implements certain provisions of the Real Estate Settlement Procedures Act of 1974, as amended (12 U.S.C. 2601et seq.). In addition, this part implements certain provisions of the Financial Institutions Reform, Recovery, and Enforcement Act, as amended (12 U.S.C. 3331et seq.). The Bureau's information-collection requirements contained in this part have been approved by the Office of Management and Budget (OMB) under the provisions of44 U.S.C. 3501et seq.and have been assigned OMB No. 3170–0015 (Truth in Lending).",
 "(b)Purpose.The purpose of this part is to promote the informed us

In [ ]:


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary = summarizer(reg_z_text_list, max_length=130, min_length=30, do_sample=False)
print(summary)

[{'summary_text': 'This part, known as Regulation Z, is issued by the Bureau of Consumer Financial Protection to implement the Federal Truth in Lending Act. This part also implements title XII, section 1204 of the Competitive Equality Banking Act of 1987 (Pub. L. 100–86, 101 Stat. 552)'}, {'summary_text': 'The purpose of this part is to promote the informed use of consumer credit by requiring disclosures about its terms and cost. The regulation does not generally govern charges for consumer credit.'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Visit CNN.com/Travel each week for a new gallery of snapshots.'}, {'summary_text': 'This part applies to each individual or business that offers or extends credit, other than a person excluded from coverage of this part by section 1029 of the Consumer Financial Protection Act of 2010. (1)'}, {'summary_text': 'The credit is offered or extended to consumers;. (i) The credit is given or extended for free to consumers. (ii) Consumers can use the credit to pay for goods and services.'}, {'summary_text': 'The offering or extension of credit is done regularly;. (ii) The offering or extending of credit should be done regularly. (i) Credit should be extended on a regular basis.'}, {'summary_text': 'The credit is subject to a finance charge or is payable by a written agreement in more than four installments; and. The credit is payable in installments of up to $10,000.'}, {'summary_text': 'The credit is primarily for personal, family, or household purposes. (iv) The credit is for personal use only. It is not intended to be used for commercial purposes.'}, {'summary_text': 'If a credit card is involved, however, certain provisions apply even if the credit is not subject to a finance charge. These provisions apply if thecredit is not payable by a written agreement in more than four installments.'}, {'summary_text': 'In addition, certain requirements of§ 1026.40apply to persons who are not creditors but who provide applications for home-equity plans to consumers. (3)'}, {'summary_text': '§ 1026.57apply to institutions of higher education. (4)Furthermore, certain requirements of§ 10 26.57 apply to higher education institutions. (5)'}, {'summary_text': 'No person is required to provide the disclosures required by sections 128(a)(16) through (19), 128(b)(4), 129C(f)(1) and (3), 129D(h) of the Truth in Lending Act. Except in transactions subject to.§ 1026.39(d)(5), no person becoming a creditor with respect to an existing residential mortgage loan is required.'}, {'summary_text': 'The regulation is divided into subparts and appendices as follows:. (d)Organization. (e) Rules and regulations. (f) Rules, regulations and rules.'}, {'summary_text': 'Subpart A contains general information. (1) Subpart A sets forth: It sets forth the purpose of this section. (2) It sets out the objectives of the section.'}, {'summary_text': 'The authority, purpose, coverage, and organization of the regulation;. (i) The authority and purpose of this regulation. (ii) The purpose, purpose and coverage of thisregulation;.'}, {'summary_text': 'The definitions of basic terms;. (ii) The definitions of some of the most common terms in the English language. (iii) The definition of some common terms, such as the word ‘fuzzy’.'}, {'summary_text': "The transactions that are exempt from coverage; and. (iii) the transactions that aren't covered by the insurance policy at the time of the transaction."}, {'summary_text': 'The method of determining the finance charge. (iv) The method of deciding the finance charges. (v) The way in which the charge is calculated. (vi)'}, {'summary_text': 'Subpart B contains the rules for open-end credit. It requires that account-opening disclosures and periodic statements be provided. It also describes special rules that apply to credit card transactions.'}, {'summary_text': 'Subpart C relates to closed-end credit. It contains rules on disclosures, treatment of credit balances, annual percentage rate calculations, rescission requirements, and advertising.'}, {'summary_text': 'Subpart D contains rules on oral disclosures, disclosures in languages other than English, record retention, effect on state laws, state exemptions, and rate limitations.'}, {'summary_text': 'Section 1026.32requires certain disclosures and provides limitations for closed-end credit transactions and open-endCredit plans that have rates or fees above specified amounts or certain prepayment penalties. (5)Subpart E contains special rules for mortgage transactions.'}, {'summary_text': 'Subpart F relates to private education loans. It contains rules on disclosures, limitations on changes in terms after approval, and the right to cancel the loan.'}, {'summary_text': "Subpart G relates to credit card accounts under an open-end (not home-secured) consumer credit plan. Section 1026.51contains rules on evaluation of a consumer's ability to make the required payments under the terms of an account."}, {'summary_text': 'Several appendices contain information such as the procedures for determinations about state laws, state exemptions and issuance of official interpretations. Special rules for certain kinds of credit plans, and the rules for computing annual percentage rates in closed-end credit transactions.'}, {'summary_text': 'Section 108 of the Truth in Lending Act contains the administrative enforcement provisions for that Act. Section 1204(c) of title XII of the Competitive Equality Banking Act of 1987, Public Law 100–86, 101 Stat. 552, incorporates by reference administrative enforcement and civil liability provisions.'}, {'summary_text': "A creditor shall credit a payment to the consumer's account as of the date of receipt. A delay in crediting does not result in a finance or other charge or except as provided inparagraph (b)of this section."}, {'summary_text': ' (b)Specific requirements for payments. (c) Specific requirements for how payments are made. (d) A specific amount of money to be paid each month.'}, {'summary_text': 'A creditor may specify reasonable requirements for payments that enable most consumers to make conforming payments. (1) General rule. (2) Creditors may not require a payment to be made in full or in part.'}, {'summary_text': 'Reasonable requirements for making payment may include:. (1) Examples of reasonable requirements for payments. (2) examples ofreasonable requirements for paying.'}, {'summary_text': 'Requiring that payments be accompanied by the account number or payment stub;. (i) Requiring that Payments be accompanied with a payment stub.'}, {'summary_text': 'Setting reasonable cut-off times for payments to be received by mail, by electronic means, by telephone, and in person (except as provided inparagraph (b)(3)of this section), provided that such cut- off times shall be no earlier than 5 p.m. on the payment due date. (ii)'}, {'summary_text': 'Specifying that only checks or money orders should be sent by mail. (iii) Specifying only checks and money orders can be sent in the mail.'}, {'summary_text': 'Specifying that payment is to be made in U.S. dollars; or. (iv) Specifying that the payment be in cash.'}, {'summary_text': 'Specifying one particular address for receiving payments, such as a post office box. (v) Specifying one specific address to receive payments.'}, {'summary_text': 'In-person payments on credit card accounts. (3) In-person payment in person. (2) In person payments in person on credit cards. (1)'}, {'summary_text': 'Notwithstanding§ 1026.10(b), payments on a credit card account under an open-end (not home-secured) consumer credit plan made in person at a branch or office of a card issuer that is a financial institution shall be considered received on the date on which the consumer makes the payment. A card issuer may impose a cut-off time earlier than 5 p.m.'}, {'summary_text': 'For purposes ofparagraph (b)(3)of this section, “financial institution” shall mean a bank, savings association, or credit union. (ii)Financial institution.'}, {'summary_text': 'Nonconforming payments. (4) nonconformity. (5) non-compliance. (6)\xa0non-conforming. payments.'}, {'summary_text': 'If a creditor specifies, on or with the periodic statement, requirements for the consumer to follow in making payments as permitted under this§ 1026.10, but accepts a payment that does not conform to the requirements, the creditor shall credit the payment.'}, {'summary_text': 'If a creditor promotes a method for making payments, such payments shall be considered conforming payments in accordance with thisparagraph. (ii)Payment methods promoted by creditor.'}, {'summary_text': "If a creditor fails to credit a payment, as required byparagraphs (a)or(b)of this section, in time to avoid the imposition of finance or other charges, the creditor shall adjust the consumer's account. (c)Adjustment of account."}, {'summary_text': "Crediting of payments when creditor does not receive or accept payments on due date. (d) Crediting when creditor doesn't receive or accepts payments on time. (e)Creditors who do not accept or receive payments on deadline."}, {'summary_text': 'If a creditor does not receive or accept payments by mail on the due date for payments, the creditor may generally not treat a payment received the next business day as late for any purpose. For purposes of thisparagraph (d), the “next business day” means the next day on which the creditor accepts or receives payments bymail.'}, {'summary_text': 'If a creditor accepts or receives payments made on the due date by a method other than mail, the creditor is not required to treat a payment made by that method on the next business day as timely.'}, {'summary_text': 'For credit card accounts under an open-end (not home-secured) consumer credit plan, a creditor may not impose a separate fee to allow consumers to make a payment by any method. For purposes ofparagraph (e)of this section, the term “creditor” includes a third party that collects, receives, or processes payments on behalf of a creditor.'}, {'summary_text': 'If a card issuer makes a material change in the address for receiving payments or procedures for handling payments, and such change causes a material delay in the crediting of a payment, the card issuer may not impose any late fee or finance charge for a late payment.'}, {'summary_text': 'When a credit balance in excess of $1 is created on a credit account, the creditor shall transmit the funds to the consumer. (a)Credit balances.'}, {'summary_text': "Credit the amount of the credit balance to the consumer's account. (1) Credit the amount to the customer's account; (2) Credit to the person's bank account."}, {'summary_text': 'Refund any part of the remaining credit balance within seven business days from receipt of a written request from the consumer;. (2) Refund any of the remainder of the credit balance in less than seven days.'}, {'summary_text': "Make a good faith effort to refund to the consumer by cash, check, or money order, or credit to a deposit account of the consumer. No further action is required if the consumer's current location is not known to the creditor."}, {'summary_text': 'Account termination. (b) Account termination (c) Accounts terminated. (d) Account terminated.Account termination (e) Accounts termination (f) Accountstermination.'}, {'summary_text': "A creditor shall not terminate an account prior to its expiration date solely because the consumer does not incur a finance charge. (1) A creditor may not terminate a customer's account if the customer does not\xa0 incur a\xa0finance charge."}, {'summary_text': 'Nothing inparagraph (b)(1)of this section prohibits a creditor from terminating an account that is inactive for three or more consecutive months. An account is inactive if no credit has been extended (such as by purchase, cash advance or balance transfer) and if it has no outstanding balance.'}, {'summary_text': 'Timely settlement of estate debts is key to a successful estate settlement. (c)Timely Settlement of Estate debts is a key to successful estate settlements.'}, {'summary_text': 'General rule. (1) General rule is that no one person can have more than one name. (2) No one can be more than two names. (3) No person can be less than three names.'}, {'summary_text': 'For credit card accounts under an open-end (not home-secured) consumer credit plan, card issuers must adopt reasonable written policies. (i)Reasonable policies and procedures required.'}, {'summary_text': 'Paragraph (c) of this section does not apply to the account of a deceased consumer if a joint accountholder remains on the account. (ii)Application to joint accounts.'}, {'summary_text': 'Timely statement of balance. (2)Timely statements of balance— (2), (3), (4), (5), (6), (7)'}, {'summary_text': "A card issuer must provide the administrator of an estate with the amount of the balance on a deceased consumer's account in a timely manner. (i)Requirement."}, {'summary_text': 'For purposes ofparagraph (c)(2)(i)of this section, providing the amount of the balance on the account within 30 days of receiving the request is deemed to be timely. (ii)Safe harbor.'}, {'summary_text': 'Limitations after receipt of request from administrator. (3) Limitations after receiving of request for information. (4) Limitation after receiving request for data. (5)'}, {'summary_text': "After receiving a request from the administrator of an estate for the amount of the balance on a deceased consumer's account, a card issuer must not impose any fees on the account."}, {'summary_text': 'A card issuer must waive or rebate any additional finance charge due to a periodic interest rate if payment in full is received within 30 days after disclosure. (ii)Limitation on trailing or residual interest.'}, {'summary_text': 'No credit card shall be issued to any person except for business, commercial, or agricultural use. (a)Issuance of credit cards.'}, {'summary_text': 'In response to an oral or written request or application for the card; or. (1) in response to a written request for a card.'}, {'summary_text': 'As a renewal of, or substitute for, an accepted credit card. (2) As a replacement for, or renews, a credit card that is not accepted by the bank.'}, {'summary_text': 'Liability of cardholder for unauthorized use. (b)Liability for use of card in violation of terms of contract. (c) Liability for misuse of card.'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Visit CNN.com/Travel each week for a new gallery of snapshots from around the world.'}, {'summary_text': 'For purposes of this section, “unauthorized use” means the use of a credit card by a person, other than the cardholder, who does not have actual, implied, or apparent authority for such use. (i)Definition of unauthorized use.'}, {'summary_text': 'The liability of a cardholder for unauthorized use of a credit card shall not exceed the lesser of $50 or the amount of money, property, labor, or services obtained by the unauthorized use. (ii)Limitation on amount.'}, {'summary_text': 'A cardholder shall be liable for unauthorized use of a credit card only if:. (2)Conditions of liability. A cardholder is liable if he or she uses the credit card without authorization.'}, {'summary_text': 'The credit card is an accepted credit card;. (i) The credit card can be used to pay for goods and services. (ii) It can also be used as a way to avoid paying for things.'}, {'summary_text': "The card issuer has provided adequate notice of the cardholder's maximum potential liability and of means by which the card issuer may be notified of loss or theft. The cardholder may give oral or written notification, and shall describe a means of notification."}, {'summary_text': 'The card issuer has provided a means to identify the cardholder on the account or the authorized user of the card. (ii) The card issuer is providing a way for the user to identify themselves. (iii) The authorized user is the card issuer.'}, {'summary_text': 'Notification to a card issuer is given when steps have been taken to provide the card issuer with the pertinent information about the loss, theft, or possible unauthorized use of a credit card.'}, {'summary_text': 'If state law or an agreement between a cardholder and the card issuer imposes lesser liability than that provided in this paragraph, the lesser liability shall govern. (4)Effect of other applicable law or agreement.'}, {'summary_text': 'If 10 or more credit cards are issued by one card issuer for use by the employees of an organization, this section does not prohibit the card issuer and the organization from agreeing to liability for unauthorized use without regard to this section. However, liability may be imposed on an employee of the organization, by either the card issuers or the organization. (5) Business use of credit cards.'}, {'summary_text': 'Right of cardholder to assert claims or defenses against card issuer. (c) Right of cardholders to assert claim or defense against card issuers.'}, {'summary_text': 'When a person who honors a credit card fails to resolve satisfactorily a dispute as to property or services purchased with the credit card in a consumer credit transaction, the cardholder may assert against the card issuer all claims (other than tort claims) and defenses arising out of the transaction and relating to the failure to resolve the dispute.'}, {'summary_text': 'If a cardholder withholds payment of the amount of credit outstanding for the disputed transaction, the card issuer shall not report that amount as delinquent until the dispute is settled or judgment is rendered.'}, {'summary_text': 'Limitations of the study were noted by the researchers. The study was conducted over a period of three months. The results were published in the June edition of the book.'}, {'summary_text': 'The rights stated inparagraphs (c)(1)and(c)(2)of this section apply only if:. (i) General.'}, {'summary_text': 'The cardholder has made a good faith attempt to resolve the dispute with the person honoring the credit card. (A) The card holder has made an attempt to settle the dispute; and.'}, {'summary_text': "The amount of credit extended to obtain the property or services that result in the assertion of the claim or defense by the cardholder exceeds $50. The disputed transaction occurred in the same state as the cardholders' current designated address or within 100 miles from that address."}, {'summary_text': 'The limitations stated inparagraph (c)(3)(i)(B)of this section shall not apply when the person honoring the credit card:. (ii)Exclusion.'}, {'summary_text': 'Is the same person as the card issuer;. (A) Is the sameperson as the credit card issuer, or. (B) Is a different person than the person who issued the card.'}, {'summary_text': 'Is controlled by the card issuer directly or indirectly;. (B) is controlled by a bank or bank-owned company;. or. (C) Is controlled by an individual or a corporation.'}, {'summary_text': 'The card is under the direct or indirect control of a third person that also directly or indirectly controls the card issuer;. The card issuer is not directly or indirect controlled by the third person.'}, {'summary_text': 'Controls the card issuer directly or indirectly;. (D) Controls the bank or bank-card issuer directly, indirectly, or through a third party;.'}, {'summary_text': "Is a franchised dealer in the card issuer's products or services; or. (E) is a franchise dealer in a product or service."}, {'summary_text': "Has obtained the order for the disputed transaction through a mail solicitation made or participated in by the card issuer. (F) Has gotten the order through the mail solicitation through a card issuer's mail solicitation."}, {'summary_text': 'Offsets by card issuer prohibited. (d) Offsets by bank or bank-card issuer prohibited by law. (e) Offset by card issuers prohibited by federal law.'}, {'summary_text': "A card issuer may not take any action, either before or after termination of credit card privileges, to offset a cardholder's indebtedness arising from a consumer credit transaction."}, {'summary_text': 'Thisparagraph (d)does not alter or affect the right of a card issuer acting under state or Federal law to do any of the following: Obtain or enforce a consensual security interest in the funds; attach or otherwise levy upon the funds.'}, {'summary_text': 'Periodic deductions. (3) per year. (2) per person. (1) per city. (4) per state. (5) per individual. (6) per municipality. (7) per town. (8) per county.'}, {'summary_text': "§ 1026.13(d)(1) does not prohibit a plan, if authorized in writing by the cardholder, under which the card issuer may periodically deduct all or part of the cardholders' credit card debt."}, {'summary_text': 'With respect to a covered separate credit feature accessible by a hybrid prepaid-credit card as defined in§ 1026.61, for purposes of thisparagraph (d)(3), “periodically” means no more frequently than once per calendar month.'}, {'summary_text': 'Prompt notification of returns and crediting of refunds. (e) Prompt notification, crediting, and credit for returns and refunds.'}, {'summary_text': "When a creditor other than the card issuer accepts the return of property or forgives a debt for services that is to be reflected as a credit to the consumer's credit card account, that creditor shall, within 7 business days from accepting the return or forgiving the debt, transmit a credit statement. (1)"}, {'summary_text': "The card issuer shall, within 3 business days from receipt of a credit statement, credit the consumer's account with the amount of the refund. (2)"}, {'summary_text': 'This section does not require refunds for returns nor does it prohibit refunds in kind. (3) If a creditor other than a card issuer routinely gives cash refunds to consumers paying in cash, the creditor shall also give credit or cash refund to consumers using credit cards.'}, {'summary_text': 'No card issuer may, by contract or otherwise, offer discounts or tie-in arrangements. (f)Discounts; tie- in arrangements.'}, {'summary_text': 'Prohibit any person who honors a credit card from offering a discount to a consumer to induce the consumer to pay by cash, check, or similar means. (1)'}, {'summary_text': "Require any person who honors the card issuer's credit card to open or maintain any account. If maintenance of an account for clearing purposes is determined to be essential to the operation of the credit card plan, it may be required only if no service charges or minimum balance requirements are imposed."}, {'summary_text': 'For guidance on whether Regulation Z or Regulation E applies in instances involving both credit and electronic fund transfer aspects, refer to Regulation E,12 CFR 1005.12(a)regarding issuance and liability for unauthorized use. This section applies to the credit aspects of combined credit/electronic fund transfer transactions, as applicable.'}, {'summary_text': 'Definition of billing error. (a) For purposes of this section, the term billing error means:. (b) A billing error is an error that occurs when a customer tries to pay for a service that is not available.'}, {'summary_text': "A reflection on or with a periodic statement of an extension of credit that is not made to the consumer. A person who has actual, implied, or apparent authority to use the consumer's credit card or open-end credit plan."}, {'summary_text': 'A reflection on or with a periodic statement of an extension of credit that is not identified in accordance with the requirements of§§ 1026.7(a)(2)or(b)(2), as applicable. (2)'}, {'summary_text': "A reflection on or with a periodic statement of an extension of credit for property or services not accepted by the consumer or the consumer's designee. (3)"}, {'summary_text': "A reflection on a periodic statement of the creditor's failure to credit properly a payment or other credit issued to the consumer's account. (4)"}, {'summary_text': 'A reflection on a periodic statement of a computational or similar error of an accounting nature that is made by the creditor. (5) A reflection on an accounting error of a mathematical or similar nature.'}, {'summary_text': 'A reflection on a periodic statement of an extension of credit for which the consumer requests additional clarification, including documentary evidence. (6) A. A. statement of a credit that the consumer asks for additional clarification. (7) A reflection on an extension that a consumer requests for further clarification.'}, {'summary_text': "The creditor's failure to mail or deliver a periodic statement to the consumer's last known address if that address was received by the creditor, in writing, at least 20 days before the end of the billing cycle. (7)"}, {'summary_text': 'Billing error notice. (b) A billing error notice is a written notice from a consumer that they have received a bill that is not what it appears to be.'}, {'summary_text': 'Creditors must send the bill to the address disclosed under§ 1026.7(a)(9)or(b)(9), as applicable, no later than 60 days after the creditor transmitted the first periodic statement that reflects the alleged billing error.'}, {'summary_text': "Enables the creditor to identify the consumer's name and account number; and. (2) Enables the consumer to identify their name and bank account number."}, {'summary_text': "To the extent possible, indicates the consumer's belief and the reasons for the belief that a billing error exists, and the type, date, and amount of the error. (3)"}, {'summary_text': 'Time for resolution; general procedures. (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2022.'}, {'summary_text': 'The creditor shall mail or deliver written acknowledgment to the consumer within 30 days of receiving a billing error notice. The creditor must comply with the appropriate resolution procedures of this section, as applicable, within the 30-day period.'}, {'summary_text': 'The creditor shall comply with the appropriate resolution procedures of this section, as applicable, within 2 complete billing cycles (but in no event later than 90 days) after receiving a billing error notice. (2)'}, {'summary_text': 'Until a billing error is resolved underparagraph (e)or(f) of this section, the following rules apply. (d)Rules pending resolution.'}, {'summary_text': 'The consumer need not pay (and the creditor may not try to collect) any portion of any required payment that the consumer believes is related to the disputed amount (including related finance or other charges). If the cardholder has enrolled in an automatic payment plan offered by the card issuer and has agreed to pay the credit card indebtedness.'}, {'summary_text': "Adverse credit reports prohibited. Creditor or its agent shall not make or threaten to make an adverse report to any person about the consumer's credit standing."}, {'summary_text': "A creditor may not accelerate any part of the consumer's indebtedness or restrict or close a consumer's account solely because the consumer has exercised in good faith rights provided by this section. (3)Acceleration of debt and restriction of account prohibited."}, {'summary_text': "A creditor is not prohibited from taking action to collect any undisputed portion of the item or bill. A creditor can deduct any disputed amount and related finance or other charges from the consumer's credit limit on the account."}, {'summary_text': 'If a creditor determines that a billing error occurred as asserted, it shall within the time limits inparagraph (c)(2)of this section. (e)Procedures if billing error occurs as asserted.'}, {'summary_text': "Correct the billing error and credit the consumer's account with any disputed amount and related finance or other charges, as applicable. (1)"}, {'summary_text': 'Mail or deliver a correction notice to the consumer. (2) Mail or deliver an apology to theconsumer. (3) If you have a problem with the site, please email us at: editorial@dailymail.co.uk.'}, {'summary_text': 'If a creditor determines that no billing error occurred, the creditor shall within the time limits inparagraph (c)(2)of this section. (f)Procedures if different billing error or no billingerror occurred.'}, {'summary_text': "Mail or deliver to the consumer an explanation that sets forth the reasons for the creditor's belief that the billing error alleged by the consumer is incorrect in whole or in part."}, {'summary_text': "Furnish copies of documentary evidence of the consumer's indebtedness, if the consumer so requests; and. (2) Furnish evidence of a consumer's debt, if they so request."}, {'summary_text': "If a different billing error occurred, correct the billing error and credit the consumer's account with any disputed amount and related finance or other charges. (3)"}, {'summary_text': "Creditor's rights and duties after resolution. If a creditor determines that a consumer owes all or part of the disputed amount and related finance or other charges, the creditor:"}, {'summary_text': 'Shall promptly notify the consumer in writing of the time when payment is due and the portion of the disputed amount and related finance or other charges that the consumer still owes.'}, {'summary_text': 'Shall allow any time period disclosed under§ 1026.6(a)(1)or(b)(2)(v), as applicable, during which the consumer can pay the amount due underparagraph (g) of this section without incurring additional finance or other charges.'}, {'summary_text': 'May report an account or amount as delinquent because the amount due underparagraph (g)(1)of this section remains unpaid after the creditor has allowed any time period disclosed under. (3)'}, {'summary_text': 'May not report that an amount or account is delinquent because the amount due under paragraph (g)(1) of the section remains unpaid, if the creditor receives further written notice from the consumer that any portion of the billing error is still in dispute. (4)'}, {'summary_text': 'Promptly reports that the amount or account is in dispute;. (i) Promptly announces that the money or account in dispute is being disputed;.'}, {'summary_text': 'Mails or delivers to the consumer (at the same time the report is made) a written notice of the name and address of each person to whom the creditor makes a report.'}, {'summary_text': 'Promptly reports any subsequent resolution of the reported delinquency to all persons to whom the creditor has made a report. (iii)'}, {'summary_text': 'A creditor that has fully complied with the requirements of this section has no further responsibilities under this section. (h)Reassertion of billing error. A consumer reasserts substantially the same billing error to a creditor.'}, {'summary_text': 'A creditor shall comply with the requirements of Regulation E,12 CFR 1005.11, and1005.18(e)as applicable, governing error resolution rather than those ofparagraphs (a),(b),(c),(e),(f), and(h)of this section.'}, {'summary_text': "An extension of credit that is incident to an electronic fund transfer occurs under an agreement between the consumer and a financial institution to extend credit when the consumer's account is overdrawn or to maintain a specified minimum balance."}, {'summary_text': 'An extension of credit that is incident to an electronic fund transfer occurs when the hybrid prepaid-credit card accesses both funds in the asset feature of the prepaid account and a credit extension with respect to a particular transaction.'}, {'summary_text': 'The annual percentage rate is a measure of the cost of credit, expressed as a yearly rate. An error in disclosure of the annual percentage rates or finance charge shall not, in itself, be considered a violation of this part.'}, {'summary_text': 'The error resulted from a corresponding error in a calculation tool used in good faith by the creditor. (1) The error was caused by a mistake in the calculation tool.'}, {'summary_text': 'Upon discovery of the error, the creditor promptly discontinues use of that calculation tool for disclosure purposes, and notifies the Bureau in writing. (2)'}, {'summary_text': 'Annual percentage rate—in general. (b)Where one or more periodic rates may be used to compute the finance charge, the annual percentage rate(s) to be disclosed for purposes of§§ 1026.60,1026.40,10 26.6, and 10 26.55.'}, {'summary_text': 'Creditors offering an open-end plan subject to the requirements of§ 1026.40need not disclose an effective annual percentage rate. Such a creditor may, at its option, disclose aneffective annual Percentage Rate.'}, {'summary_text': "If the finance charge is determined solely by applying one or more periodic rates, at the creditor's option, either:. (1)Solely periodic rates imposed."}, {'summary_text': 'By multiplying each periodic rate by the number of periods in a year; or. (i) by the total number of years in the year. (ii) by multiplying each period by the annual number of months.'}, {'summary_text': 'By dividing the total finance charge for the billing cycle by the sum of the balances to which the periodic rates were applied and multiplying the quotient (expressed as a percentage) by the number of billing cycles in a year.'}, {'summary_text': 'If the finance charge imposed during the billing cycle is or includes a minimum, fixed, or other charge not due to the application of a periodic rate, other than a charge with respect to any specific transaction, an annual percentage rate cannot be determined under this section. (2)Minimum or fixed charge, but not transaction charge, imposed.'}, {'summary_text': 'The annual percentage rate shall not be less than the largest rate determined by multiplying each periodic rate imposed during the billing cycle by the number of periods in a year. If the finance charge is or includes a loan fee, points, or similar charge that relates to the opening, renewing, or continuing an account, the amount of such charge will not be included in the calculation. See appendix F to this part regarding determination of the denominator of the fraction.'}, {'summary_text': "If the finance charge imposed during the billing cycle is or includes a minimum, fixed, or other charge not due to the application of a periodic rate and the total finance charge does not exceed 50 cents for a monthly or longer billing cycle, or the pro rata part of 50 cents, at the creditor's option, the charge is multiplied by the number of periods in a year."}, {'summary_text': 'If the provisions ofparagraph (c)(1)(ii)or(c)(2)of this section apply and all or a portion of the finance charge is determined by the application of one or more daily periodic rates, the annual percentage rate may be determined. (d)Calculations where daily periodic rate applied.'}, {'summary_text': 'By dividing the total finance charge by the average of the daily balances and multiplying the quotient by the number of billing cycles in a year. (1)'}, {'summary_text': 'By dividing the total finance charge by the sum of the daily balances and multiplying the quotient by 365. (2) By dividing thetotal finance charge. by theSum of the Daily balances. (3) by theQuotient of 365.'}, {'summary_text': "Consumer's right to rescind. (a)Consumer's rights to a refund. (b) Consumer rights to an apology. (c) Consumer right to an explanation. (d) Consumers right to a full refund."}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Visit CNN.com/Travel each week for a new gallery of snapshots from around the world.'}, {'summary_text': 'Each consumer whose ownership interest is or will be subject to the security interest shall have the right to rescind: each credit extension made under the plan; the plan when the plan is opened; a security interest when added or increased.'}, {'summary_text': 'The consumer does not have the right to rescind each credit extension made under the plan if such extension is made in accordance with a previously established credit limit for the plan. (ii)'}, {'summary_text': 'To exercise the right to rescind, the consumer shall notify the creditor of the rescission by mail, telegram, or other means of written communication. Notice is considered given when mailed, or when filed for telegraphic transmission.'}, {'summary_text': "The consumer may exercise the right to rescind until midnight of the third business day following the occurrence described inparagraph (a)(1) of this section. If the required notice and material disclosures are not delivered, theRight to rescind shall expire 3 years after the occurrence giving rise to the right of rescission, or upon transfer of all of the consumer's interest in the property."}, {'summary_text': 'When more than one consumer has the right to rescind, the exercise of the right by one consumer shall be effective as to all consumers. (4)'}, {'summary_text': 'Creditor shall deliver two copies of the notice of the right to rescind to each consumer entitled to rescind. The notice shall identify the transaction or occurrence and clearly and conspicuously disclose the following.'}, {'summary_text': "The retention or acquisition of a security interest in the consumer's principal dwelling. (1) The retention or. acquisition of an interest in a home. (2) The purchase of a residence."}, {'summary_text': "The consumer's right to rescind, as described inparagraph (a)(1) of this section. (2) The consumer'sright to rescind."}, {'summary_text': "How to exercise the right to rescind, with a form for that purpose, designating the address of the creditor's place of business. (3)"}, {'summary_text': 'The effects of rescission, as described inparagraph (d)of this section. (4) The effects of the rescission of this section are described in paragraph (D) of thissection.'}, {'summary_text': 'The date the rescission period expires. (5) The date of the end of the rescue plan. (6) The day the plan is completed. (7)'}, {'summary_text': "Unless a consumer waives the right to rescind underparagraph (e)of this section, no money shall be disbursed other than in escrow, no services shall be performed, and no materials delivered until after the rescission period has expired. (c)Delay of creditor's performance."}, {'summary_text': 'Effects of rescission. (d) effects of rescissions. (e) effects on the environment. (f) Effects on the economy. (g)effects on the health of the population.'}, {'summary_text': 'When a consumer rescinds a transaction, the security interest giving rise to the right of rescission becomes void. The consumer shall not be liable for any amount, including any finance charge.'}, {'summary_text': 'Within 20 calendar days after receipt of a notice of rescission, the creditor shall return any money or property that has been given to anyone in connection with the transaction and shall take any action necessary to reflect the termination of the security interest.'}, {'summary_text': "Tender of money must be made at the creditor's designated place of business. If the creditor does not take possession of the money or property within 20 calendar days after the consumer's tender, the consumer may keep it without further obligation."}, {'summary_text': 'The procedures outlined inparagraphs (d)(2)and(3)of this section may be modified by court order. (4)'}, {'summary_text': 'Consumer may modify or waive the right to rescind if the consumer determines that the extension of credit is needed to meet a bona fide personal financial emergency. The consumer shall give the creditor a dated written statement that describes the emergency.'}, {'summary_text': 'The right to rescind does not apply to the following:. (f)Exempt transactions. (e) Exempt transactions in which the owner of the property is not entitled to rescind.'}, {'summary_text': 'A residential mortgage transaction. (1) A mortgage transaction with a bank. (2) A residential mortgage with a lender. (3) Two residential mortgage transactions with a banker.'}, {'summary_text': "A credit plan in which a state agency is a creditor. (2) A credit plan where the state is a creditors. (3) A state agency's credit plan with a state-owned bank."}, {'summary_text': 'If an advertisement for credit states specific credit terms, it shall state only those terms that actually are or will be arranged or offered by the creditor. (a)Actually available terms.'}, {'summary_text': '’s of terms that require additional disclosures. (b) ‘Extra disclosure’ is a term used to refer to a person who has provided additional information about his or her financial situation.'}, {'summary_text': 'Any term required to be disclosed under§ 1026.6(b)(3)set forth affirmatively or negatively in an advertisement for an open-end (not home-secured) credit plan triggers additional disclosures under this section. The advertisement shall also clearly and conspicuously set forth the following.'}, {'summary_text': 'Any minimum, fixed, transaction, activity or similar charge that is a finance charge under§ 1026.4that could be imposed. (i)'}, {'summary_text': 'Any periodic rate that may be applied expressed as an annual percentage rate as determined under§ 1026.14(b). If the plan provides for a variable periodic rate, that fact shall be disclosed.'}, {'summary_text': 'Any membership or participation fee that could be imposed. (iii) Any membership or Participation Fee. (ii) Any Membership or Participation Fees. ( iv) Any Participation Fee that could Be imposed.'}, {'summary_text': 'If an advertisement for credit to finance the purchase of goods or services specified in the advertisement states a periodic payment amount, the advertisement shall also state the total of payments and the time period to repay the obligation. (2)'}, {'summary_text': 'Catalogs or other multiple-page advertisements; electronic advertisements. (c) Catalogs and otherMultiplePageAds.com are owned by The Associated Press, a division of Time Warner Inc.'}, {'summary_text': 'If a catalog or other multiple-page advertisement, or an electronic advertisement, gives information in a table or schedule in sufficient detail to permit determination of the disclosures required by paragraph (b) of this section, it shall be considered a single advertisement.'}, {'summary_text': 'The table or schedule is clearly and conspicuously set forth; and. (i) The table ordule is clearly, conspicuously, set forth.'}, {'summary_text': 'Any statement of terms set forth in§ 1026.6appearing anywhere else in the catalog or advertisement clearly refers to the page or location where the table or schedule begins. (ii)'}, {'summary_text': 'A catalog or other multiple-page advertisement or an electronic advertisement (such as an advertisement appearing on an Internet Web site) complies with this paragraph if the table or schedule of terms includes all appropriate disclosures.'}, {'summary_text': 'Additional requirements for home-equity plans. (d) Additional requirements for homes with more than one home owner. (e) More requirements for homeowners with multiple homes. (f) More Requirements for home owners with multiple properties.'}, {'summary_text': 'If any of the terms required to be disclosed under§ 1026.6(a)(1)or(a)or the payment terms of the plan are set forth, affirmatively or negatively, in an advertisement for a home-equity plan, the advertisement also shall clearly and conspicuously set forth the following.'}, {'summary_text': 'Any loan fee that is a percentage of the credit limit under the plan and an estimate of any other fees imposed for opening the plan, stated as a single dollar amount or a reasonable range.'}, {'summary_text': 'Any periodic rate used to compute the finance charge, expressed as an annual percentage rate as determined under§ 1026.14(b). (ii)'}, {'summary_text': 'The maximum annual percentage rate that may be imposed in a variable-rate plan. (iii) The maximum annual Percentage Rate that can be imposed on a variable rate plan.'}, {'summary_text': 'If an advertisement states an initial annual percentage rate that is not based on the index and margin used to make later rate adjustments in a variable-rate plan, the advertisement also shall state with equal prominence and in close proximity to the initial rate. (2)Discounted and premium rates.'}, {'summary_text': 'The period of time such initial rate will be in effect; and. (i) The period of the time such an initial rate is in effect.'}, {'summary_text': 'A reasonably current annual percentage rate that would have been in effect using the index and margin. (ii) A reasonably current Annual Percentage Rate.'}, {'summary_text': 'A balloon payment results if paying the minimum periodic payments does not fully amortize the outstanding balance by a specified date or time. The consumer is required to repay the entire outstanding balance at such time.'}, {'summary_text': 'That a balloon payment will result; and. (i)That a Balloon Payment will result. and (ii) That a balloon Payment will Result.'}, {'summary_text': 'The amount and timing of the balloon payment that will result if the consumer makes only the minimum payments for the maximum period of time that the consumer is permitted to make such payments. (ii)'}, {'summary_text': 'An advertisement that states that any interest expense incurred under the home-equity plan is or may be tax deductible may not be misleading in this regard. (4)'}, {'summary_text': 'The interest on the portion of the credit extension that is greater than the fair market value of the dwelling is not tax deductible for Federal income tax purposes.'}, {'summary_text': 'The consumer should consult a tax adviser for further information regarding the deductibility of interest and charges. (i) The consumer should check with their tax adviser before making any changes to their tax bill.'}, {'summary_text': 'An advertisement may not refer to a home-equity plan as “free money” or contain a similarly misleading term. (5)'}, {'summary_text': 'Promotional rates and payments. (6) Promotional rates are based on a one-year subscription to CNN.com/Impact.'}, {'summary_text': 'The following definitions apply for purposes ofparagraph (d)(6)of this section. (i)Def definitions. (ii) Definitions.'}, {'summary_text': 'The term “promotional rate” means any annual percentage rate that is not based on the index and margin that will be used to make rate adjustments under the plan. (A)Promotional rate.'}, {'summary_text': 'The term “promotional payment” means:. (B)Promotional payment. (A) Promotional payment, or ‘promotion,’ is a term used to refer to a payment.'}, {'summary_text': 'For a variable-rate plan, any minimum payment applicable for a promotional period that:. (1) For a variable rate plan, the minimum payment is $1,000. (2) The minimum payment for a variable plan is $2,500.'}, {'summary_text': 'Is not derived by applying the index and margin to the outstanding balance when such index andmargin will be used to determine other minimum payments under the plan; and. (i)'}, {'summary_text': 'Is less than other minimum payments under the plan derived by applying a reasonably current index and margin that will be used to determine the amount of such payments, given an assumed balance. (ii)'}, {'summary_text': 'For a plan other than a variable-rate plan, any minimum payment applicable for a promotional period if that payment is less than other payments required under the plan. (2)'}, {'summary_text': 'A “promotional period” means a period of time, less than the full term of the loan, that the promotional rate or promotional payment may be applicable. (C)'}, {'summary_text': 'If any annual percentage rate that may be applied to a plan is a promotional rate, or if any payment applicable to aPlan is promotional payment, the following must be disclosed in any advertisement, other than television or radio advertisements. (ii)Stating the promotional period and post-promotional rate or payments.'}, {'summary_text': 'The period of time during which the promotional rate or promotional payment will apply;. (A) The period of. time when the promotional rates or promotional payments will apply. (B) The amount of time.'}, {'summary_text': 'In the case of a promotional rate, any annual percentage rate that will apply under the plan must be disclosed in accordance with the accuracy standards in§ 1026.40or§ 10 26.16(b)(1)(ii)'}, {'summary_text': 'In the case of a promotional payment, the amounts and time periods of any payments that will apply under the plan. In variable-rate transactions, payments will be disclosed based on a reasonably current index and margin.'}, {'summary_text': 'The requirements inparagraph (d)(6)(ii)of this section do not apply to an envelope in which an application or solicitation is mailed, or to a banner advertisement or pop-up advertisement. (iii)Envelope excluded.'}, {'summary_text': 'An advertisement made through television or radio stating any of the terms requiring additional disclosures underparagraphs (b)(1)or(d)(1), as applicable, may alternatively comply with this section. (e)Alternative disclosures—television or radio advertisements.'}, {'summary_text': 'An advertisement may not refer to an annual percentage rate as “fixed,” or use a similar term. An advertisement must also specify a time period that the rate will be fixed.'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}, {'summary_text': 'The requirements of this paragraph apply to any advertisement of an open-end (not home-secured) plan, including promotional materials accompanying applications or solicitations.'}, {'summary_text': 'Definitions. (1) Definitions. (2) definitions. (3) definition. (4)definition. (5) definition of definition.'}, {'summary_text': 'Promotional ratemeans any annual percentage rate applicable to one or more balances or transactions on an open-end (not home-secured) plan for a specified period of time.'}, {'summary_text': 'Introductory ratemeans a promotional rate offered in connection with the opening of an account. (i) Introductory rate is the rate at which you pay for the service.'}, {'summary_text': 'Promotional periodmeans the maximum time period for which a promotional rate or promotional fee may be applicable. (ii) Promotional period is the period during which a certain rate or fee is available.'}, {'summary_text': 'Promotional feemeans a fee required to be disclosed under§ 1026.6(b)(1)and(2)applicable to an open-end (not home-secured) plan, or to one or more balances or transactions.'}, {'summary_text': 'Introductory feemeans a promotional fee offered in connection with the opening of an account. (v)Introductory fee is a fee offered to you as part of the process of opening your account.'}, {'summary_text': 'If any annual percentage rate or fee that may be applied to the account is an introductory rate or introductory fee, the termintroductoryorintromust be in immediate proximity to each listing. (3)Stating the term “introductory”.'}, {'summary_text': 'If any annual percentage rate that may be applied to the account is a promotional rate underparagraph (g)(2)(i) of this section, the information must be stated in a clear and conspicuous manner in the advertisement. (4) Stating the promotional period and post-promotional rate or fee.'}, {'summary_text': 'When the promotional rate or promotional fee will end. (i) When the promotional rates or promotional fees will end; (ii) When they will end, or when they will begin.'}, {'summary_text': 'The annual percentage rate that will apply after the end of the promotional period. If such rate is variable, it must comply with the accuracy standards.'}, {'summary_text': 'The fee that will apply after the end of the promotional period. (iii) The fee that would apply if you applied for a new job after the period.'}, {'summary_text': 'The requirements inparagraph (g)(4)of this section do not apply to an envelope or other enclosure in which an application or solicitation is mailed, or to a banner advertisement or pop-up advertisement. (5)Envelope excluded.'}, {'summary_text': 'Deferred interest or similar offers. (h)Deferredinterest or similar offer. (i) deferred interest or other offers; (j) deferredinterest; (k) deferred Interest.'}, {'summary_text': 'The requirements of this paragraph apply to any advertisement of an open-end credit plan not subject to§ 1026.40. This includes promotional materials accompanying applications or solicitations.'}, {'summary_text': '“Deferred interest” means finance charges, accrued on balances or transactions, that a consumer is not obligated to pay or that will be waived or refunded to a consumer. The maximum period from the date the consumer becomes obligated for the balance or transaction until the specified date by which the consumer must pay the balance in full is the “deferred interest period”'}, {'summary_text': 'If a deferred interest offer is advertised, the deferred interest period must be stated in a clear and conspicuous manner in the advertisement. If the phrase “no interest” or similar term regarding the possible avoidance of interest obligations is stated, the term “if paid in full” must also be stated.'}, {'summary_text': 'Stating the terms of the deferred interest or similar offer. If any deferred interest offer is advertised, the information must be stated in the advertisement, in language similar to Sample G–24 in appendix G.'}, {'summary_text': 'A statement that interest will be charged from the date the consumer becomes obligated for the balance or transaction subject to the deferred interest offer. (i) A statement that the consumer will pay interest if the balance is not paid in full within a deferred interest period; and.'}, {'summary_text': 'A statement, if applicable, that interest will be charged from the date the consumer incurs the balance or transaction subject to the deferred interest offer if the account is in default. (ii)'}, {'summary_text': 'The requirements inparagraph (h)(4)of this section do not apply to an envelope or other enclosure in which an application or solicitation is mailed. (5)Envelope excluded.'}, {'summary_text': 'except for the disclosures required by§ 1026.19(e),(f), and(g):. (a) Form of disclosures.'}, {'summary_text': "The creditor shall make the disclosures required by this subpart clearly and conspicuously in writing. The disclosures may include an acknowledgment of receipt, the date of the transaction, and the consumer's name, address, and account number."}, {'summary_text': "The terms “finance charge” and “annual percentage rate,” when required to be disclosed under§ 1026.18(d)and(e), shall be more conspicuous than any other disclosure, except the creditor's identity. For private education loan disclosures made in compliance with\xa01026.47, the term ‘annual Percentage Rate’ and the corresponding percentage rate must be less conspicuous than the term 'finance charges'"}, {'summary_text': 'The creditor shall make disclosures before consummation of the transaction. In certain residential mortgage transactions, special timing requirements are set forth in§ 1026.19(a). In certain variable-rate transactions, the timing of disclosures may be delayed in accordance withparagraphs (g)and(h) of this section.'}, {'summary_text': 'Basis of disclosures and use of estimates. (c) Basis of estimates: Estimates are based on information provided by the author and its sources.'}, {'summary_text': 'The disclosures shall reflect the terms of the legal obligation between the parties. (1) The disclosures shall Reflect the Terms of the Legal obligation Between the Parties.'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York City for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}, {'summary_text': 'If any information necessary for an accurate disclosure is unknown to the creditor, the creditor shall make the disclosure based on the best information reasonably available at the time the disclosure is provided to the consumer. The disclosure is an estimate.'}, {'summary_text': 'For a transaction in which a portion of the interest is determined on a per-diem basis and collected at consummation, any disclosure affected by the per- diem interest shall be considered accurate if the disclosure is based on the information known to the creditor at the time that the disclosure documents are prepared. (ii)'}, {'summary_text': 'The creditor may disregard the effects of the following in making calculations and disclosures. (3) The creditor may not use the following to make calculations or disclosures.'}, {'summary_text': 'That payments must be collected in whole cents. (i)That payments should be made in whole seconds. (ii) That they should be paid in whole dollars. (iii)'}, {'summary_text': 'That dates of scheduled payments and advances may be changed because the scheduled date is not a business day. (ii) That dates of paid-out advances and payments may be change because of a change in business hours.'}, {'summary_text': 'That months have different numbers of days. (iii) That months and days have different days in different parts of the year. (iv) that months and weeks have different dates in different months.'}, {'summary_text': 'The occurrence of leap year. (iv) The occurrence of a leap year in a given year. The number of leap years in a year, and the year in which they occur.'}, {'summary_text': 'In making calculations and disclosures, the creditor may disregard any irregularity in the first period that falls within the limits described below and any payment schedule irregularity that results from the irregular first period.'}, {'summary_text': 'For transactions in which the term is less than 1 year, a first period not more than 6 days shorter or 13 days longer than a regular period. (i)'}, {'summary_text': 'For transactions in which the term is at least 1 year and less than 10 years, a first period not more than 11 days shorter or 21 days longer than a regular period. (ii)'}, {'summary_text': 'For transactions in which the term is at least 10 years, a first period shorter than or not more than 32 days longer than a regular period. (iii)'}, {'summary_text': 'If an obligation is payable on demand, the creditor shall make the disclosures based on an assumed maturity of 1 year. If an alternate maturity date is stated in the legal obligation between the parties, the disclosures shall be based on that date.'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York City for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}, {'summary_text': 'A series of advances under an agreement to extend credit up to a certain amount may be considered as one transaction. (i) A series of loans under a credit agreement may be seen as a single transaction.'}, {'summary_text': 'When a multiple-advance loan to finance the construction of a dwelling may be permanently financed by the same creditor, the construction phase and the permanent phase may be treated as either one transaction or more than one transaction.'}, {'summary_text': 'If there is more than one consumer, the disclosures may be made to any consumer who is primarily liable on the obligation. If the transaction is rescindable under§ 1026.23, however, the disclosure shall be given to each consumer who has the right to rescind.'}, {'summary_text': 'If a disclosure becomes inaccurate because of an event that occurs after the creditor delivers the required disclosures, the inaccuracy is not a violation of this part. New disclosures may be required underparagraph (f) of this section.'}, {'summary_text': 'Creditor shall disclose before consummation (subject to the provisions of§ 1026.19(a)(2),(e), and(f)):. (f)Early disclosures.'}, {'summary_text': 'Any changed term unless the term was based on an estimate in accordance with§ 1026.17(c)(2) and was labeled an estimate.'}, {'summary_text': 'All changed terms, if the annual percentage rate at the time of consummation varies by more than 1 percentage point in a regular transaction, or more than1 percent in an irregular transaction. (2)'}, {'summary_text': 'Creditor may delay disclosures until the due date of the first payment. (g)Mail or telephone orders—delay in disclosures. Creditor receives a purchase order or a request for an extension of credit by mail, telephone, or facsimile machine.'}, {'summary_text': 'The cash price or the principal loan amount. (1) The cash price is the price of the loan. (2) The principal loan is the cash price.'}, {'summary_text': 'The total sale price. (2) The sale price of the house. (1) The price of its contents. (3) The value of the property.'}, {'summary_text': 'The finance charge. (3) The finance charge, (3), the finance charge and the finance charges. (2) The Finance Charge, (2), the Finance Charge and the Finance Charges. (1)'}, {'summary_text': 'The annual percentage rate, and if the rate may increase after consummation, the following disclosures are made. (4) The annual percentage rates are based on a percentage rate.'}, {'summary_text': 'The circumstances under which the rate may increase. (i) The circumstances in which the rates may be increased. (ii) The reasons for an increase in the rate.'}, {'summary_text': 'Any limitations on the increase. (ii) Any restrictions on the amount of money that can be raised. (iii) Any limitations on how the money can be spent.'}, {'summary_text': 'The effect of an increase. (ii) The effect of a decrease. (iii) The effects of a decline. (iv) The impact of a change.'}, {'summary_text': 'The terms of repayment. (5) The terms of repayments. (4) The amount of the loan. (3) The repayment terms. (2) The sum of the loans.'}, {'summary_text': 'Creditor may delay required disclosures until the due date of the first payment for the current sale. If a credit sale is one of a series made under an agreement, subsequent sales may be added to an outstanding balance.'}, {'summary_text': 'The consumer has approved in writing the annual percentage rate or rates, the range of balances to which they apply, and the method of treating any unearned finance charge on an existing balance.'}, {'summary_text': 'The creditor retains no security interest in any property after the creditor has received payments equal to the cash price and any finance charge attributable to the sale of that property. For purposes of this provision, in the case of items purchased on different dates, the first purchased is deemed the first item paid for.'}, {'summary_text': 'For transactions involving an interim credit extension under a student credit program for which an application is received prior to the mandatory compliance date, the creditor need not make the following disclosures. The creditor must make complete disclosures at the time the creditor and consumer agree upon the repayment schedule for the total obligation.'}, {'summary_text': '§ 1026.19(e)and(f), the creditor shall disclose the following information as applicable. For each transaction other than a mortgage transaction subject to the disclosure rules, the creditor must disclose the information.'}, {'summary_text': "The identity of the creditor making the disclosures. (a)Creditor. (b) Creditor's name. (c) Identity of creditor making disclosures."}, {'summary_text': 'The amount financed is calculated by:. (a) The amount financed,using that term, and a brief description such asthe amount of credit provided to you or on your behalf. (b) The term financed.'}, {'summary_text': 'Determining the principal loan amount or the cash price (subtracting any downpayment);. (1) Determining how much you want to pay in interest.'}, {'summary_text': ' adding any other amounts that are financed by the creditor and are not part of the finance charge. (2) Adding any other amount that is financed by a creditor and is not included in the charge.'}, {'summary_text': 'Subtracting any prepaid finance charge. (3) Subtracts any pre-paid finance charge from the amount of the loan.'}, {'summary_text': ' Itemization of amount financed. (c) Itemized amount financed: $1.4 billion. (d) Itemization: $2.2 billion.'}, {'summary_text': 'except as provided inparagraphs (c)(2)and(c)(3)of this section, a separate written itemization of the amount financed, including:. (1)'}, {'summary_text': 'The amount of any proceeds distributed directly to the consumer. (i) The amount of money that is sold to the public. (ii) The percentage of the proceeds that are distributed to the consumers.'}, {'summary_text': "The amount credited to the consumer's account with the creditor. (ii) The amount credited from the consumer to the creditor, if any."}, {'summary_text': "The following payees may be described using generic or other general terms and need not be further identified: public officials or government agencies, credit reporting agencies, appraisers, and insurance companies. (iii)Any amounts paid to other persons by the creditor on the consumer's behalf."}, {'summary_text': 'The prepaid finance charge. (iv) The pre-paid finance charge, or PFC, is a charge for prepaid loans. (v) PFCs can be used to pay for other services.'}, {'summary_text': 'The creditor need not comply withparagraph (c)(1) of this section if the creditor provides a statement that the consumer has the right to receive a written itemization of the amount financed. (2) The creditor must provide a space for the consumer to indicate whether it is desired.'}, {'summary_text': 'Good faith estimates of settlement costs provided for transactions subject to the Real Estate Settlement Procedures Act (12 U.S.C. 2601et seq.) may be substituted for the disclosures required byparagraph (c)(1)of this section.'}, {'summary_text': 'The finance charge is the dollar amount the credit will cost you. (d)Finance charge,using that term, and a brief description.'}, {'summary_text': 'In a transaction secured by real property or a dwelling, the disclosed finance charge shall be treated as accurate if the amount disclosed as the finance charge. (1)'}, {'summary_text': 'i. (i)Is understated by no more than $100; or. (ii) is understated in a way that makes it seem as if it is more than that.'}, {'summary_text': 'Is greater than the amount required to be disclosed. (ii) Is greater than  the amount of money that can be raised for charity. (iii) Is more than  $1 million.'}, {'summary_text': 'In any other transaction, the amount disclosed as the finance charge shall be treated as accurate if it is not more than $5 above or below the amount required to be disclosed. (2)'}, {'summary_text': 'Theannual percentage rate is the cost of your credit as a yearly rate. For any transaction involving a finance charge of $5 or less on an amount financed of $75 or less, the creditor need not disclose the annual percentage rate.'}, {'summary_text': ' variable rate. (f) variable rate (g) Variable rate (h) Variable Rate (i) variable Rate (j) variableRate (k) variable rates.'}, {'summary_text': "Except as provided inparagraph (f)(3)of this section, if the annual percentage rate may increase after consummation in a transaction not secured by the consumer's principal dwelling. (1)"}, {'summary_text': 'The circumstances under which the rate may increase. (i) The circumstances in which the rates may be increased. (ii) The reasons for an increase in the rate.'}, {'summary_text': 'Any limitations on the increase. (ii) Any restrictions on the amount of money that can be raised. (iii) Any limitations on how the money can be spent.'}, {'summary_text': 'The effect of an increase. (ii) The effect of a decrease. (iii) The effects of a decline. (iv) The impact of a change.'}, {'summary_text': 'An example of the payment terms that would result from an increase. (iv) An example of an increase in the amount that would be paid to the government.'}, {'summary_text': "If the annual percentage rate may increase after consummation in a transaction secured by the consumer's principal dwelling with a term greater than one year. (2)"}, {'summary_text': 'The fact that the transaction contains a variable-rate feature. (i) The fact that it is a transaction with a variable rate. (ii) A variable rate feature.'}, {'summary_text': 'A statement that variable-rate disclosures have been provided earlier. (ii) A statement that Variable-Rate Disclosure Disclosure (i) has been made earlier.'}, {'summary_text': 'Information provided in accordance with§§ 1026.18(f)(2)and1026.19(b)may be substituted for the disclosures required byparagraph (f)(1)of this section. (3)'}, {'summary_text': 'Other than for a transaction that is subject toparagraph (s)of this section, the number, amounts, and timing of payments scheduled to repay the obligation. (g)Payment schedule.'}, {'summary_text': 'In a demand obligation with no alternate maturity date, the creditor may comply with this paragraph by disclosing the due dates or payment periods of any scheduled interest payments for the first year. (1)'}, {'summary_text': 'In a transaction in which a series of payments varies because a finance charge is applied to the unpaid principal balance, the creditor may comply with this paragraph by disclosing the following information.'}, {'summary_text': 'The dollar amounts of the largest and smallest payments in the series. (i) The dollar amount of the smallest and largest payments. (ii) The amount of each payment.'}, {'summary_text': 'A reference to the variations in the other payments in the series. (i) A reference to a reference to an earlier payment. (ii)'}, {'summary_text': 'Thetotal of payments,using that term, and a descriptive explanation such as “the amount you will have paid when you have made all scheduled payments” In any transaction involving a single payment, the creditor need not disclose the total.'}, {'summary_text': 'If the obligation has a demand feature, that fact shall be disclosed. When the disclosures are based on an assumed maturity of 1 year as provided in§ 1026.17(c)(5), that fact also shall also be disclosure.'}, {'summary_text': 'In a credit sale, thetotal sale price is the sum of the cash price, the items described in paragraph (b)(2), and the finance charge disclosed underparagraph (d)of this section.'}, {'summary_text': 'Prepayment. (k) pre-payments. ( k) prepayment.com pre-paid.com. pre-payment.co.uk. Pre-payment.com Pre-Payments.com is a payment system that allows users to pay by credit card.'}, {'summary_text': "When an obligation includes a finance charge computed from time to time by application of a rate to the unpaid principal balance. A statement indicating whether or not a charge may be imposed for paying all or part of a loan's principal balance before the date on which the principal is due."}, {'summary_text': 'When an obligation includes a finance charge other than the finance charge described inparagraph (k)(1)of this section, a statement indicating whether or not the consumer is entitled to a rebate of any finance charge if the obligation is prepaid in full or in part.'}, {'summary_text': 'Any dollar or percentage charge that may be imposed before maturity due to a late payment, other than a deferral or extension charge. (l)Late payment.'}, {'summary_text': 'The fact that the creditor has or will acquire a security interest in the property purchased as part of the transaction, or in other property identified by item or type.'}, {'summary_text': 'The items required by§ 1026.4(d)in order to exclude certain insurance premiums and debt cancellation fees from the finance charge.'}, {'summary_text': 'The disclosures required by§ 1026.4(e)in order to exclude from the finance charge certain fees prescribed by law or certain premiums for insurance in lieu of perfecting a security interest.'}, {'summary_text': 'A statement that the consumer should refer to the appropriate contract document for information about nonpayment, default, the right to accelerate the maturity of the obligation, and prepayment rebates and penalties.'}, {'summary_text': 'In a residential mortgage transaction, a subsequent purchaser of the dwelling from the consumer may be permitted to assume the remaining obligation on its original terms. (q)'}, {'summary_text': 'Creditor requires consumer to maintain a deposit as a condition of the specific transaction. Annual percentage rate does not reflect the effect of the required deposit. A required deposit need not include:'}, {'summary_text': 'An escrow account for items such as taxes, insurance or repairs. (1) An escrow bank account for such items as taxes or insurance. (2) A bank account to pay for repairs or other expenses.'}, {'summary_text': 'A deposit that earns not less than 5 percent per year; or. (2) A deposit that earn not more than 5 per cent per year.'}, {'summary_text': 'Payments under the Morris Plan. (3) Payments under a Morris Plan are tax-free. (2) Tax-free money can be used to buy goods and services.'}, {'summary_text': 'Interest rate and payment summary for mortgage transactions. For a closed-end transaction secured by real property or a dwelling, other than a transaction that is subject to§ 1026.19(e)and(f), the creditor shall disclose the following information about the interest rate and payments.'}, {'summary_text': 'The information in paragraphs (s)(2)–(4) of this section shall be in the form of a table, with no more than five columns, with headings and format substantially similar to Model Clause H–4(E), H–3(F), H-4(G), or H-3(H) in appendix H to this part. The table shall be placed in a prominent location, and must be in a minimum 10-point font.'}, {'summary_text': 'Interest rates are at an all-time high. The U.S. Federal Reserve is raising interest rates for the first time in more than a decade.'}, {'summary_text': 'Amortizing loans. (i) amortizing loan payments. (ii) Amortizing debt. ( i) ammortizing loan repayments. (iii) am-mortgage.'}, {'summary_text': "For a fixed-rate mortgage, the interest rate at consummation. (A) For a variable-rate loan, the rate at the time of the loan. (B) For an adjustable-rate deal, the rates at the loan's origination."}, {'summary_text': 'For an adjustable-rate or step-rate mortgage:. (B) For a fixed rate mortgage: (A) for fixed-rate mortgages. For a variable-rate Mortgage: (C) for variable- rate mortgages.'}, {'summary_text': 'The interest rate at consummation and the period of time until the first interest rate adjustment may occur, labeled as the “introductory rate and monthly payment”. (1)'}, {'summary_text': 'The maximum interest rate that may apply during the first five years after the date on which the first regular periodic payment will be due. (2) The earliest date onWhich that rate may apply, labeled as “maximum during first 5 years”.'}, {'summary_text': 'The maximum interest rate that may apply during the life of the loan and the earliest date on which that rate may apply, labeled as “maximum ever” (3)'}, {'summary_text': 'If the loan provides for payment increases as described inparagraph (s)(3)(i)(B) of this section, the interest rate in effect at the time the first such payment increase is scheduled to occur will be labeled as “first adjustment” If the loan is an adjustable-rate mortgage or, otherwise, labeled as "first increase"'}, {'summary_text': 'Negative amortization loans. For a negative amortisation loan, the bank must have a negative credit rating. The bank must also have a positive credit rating of at least 100.'}, {'summary_text': 'The interest rate at consummation and the length of time until it will adjust, and the label “introductory” or “intro” (A)'}, {'summary_text': 'The maximum interest rate that could apply when the consumer must begin making fully amortizing payments under the terms of the legal obligation. (B)'}, {'summary_text': 'If the minimum required payment will increase before the consumer must begin making fully amortizing payments, the maximum interest rate that could apply at the time of the first payment increase and the date the increase is scheduled to occur. (C) If a second increase in theminimum required payment may occur.'}, {'summary_text': 'For an amortizing adjustable-rate mortgage, if the interest rate at consummation is less than the fully-indexed rate, placed in a box directly beneath the table required byparagraph (s)(1)of this section. Model Clause H–4(I) in appendix H to this part.'}, {'summary_text': 'The interest rate that applies at consummation and the period of time for which it applies;. (A) The interest rates that apply at. consummations and the time period for which they apply;.'}, {'summary_text': 'A statement that, even if market rates do not change, the interest rate will increase at the first adjustment. (B) A designation of the place in sequence of the month or year, as applicable, of such rate adjustment.'}, {'summary_text': "The fully-indexed rate. (C) The fully- indexed rate is based on the U.S. dollar's value against the Euro. The rate is 1.5 per cent."}, {'summary_text': 'Payments for amortizing loans. (3) Payments forAmortizing Loans. (2) Amortizing Loan Payments. (1) Amorizing Loan Payments.'}, {'summary_text': 'If all periodic payments will be applied to accrued interest and principal, for each interest rate disclosed underparagraph (s)(2)(i)of this section:. (i)Principal and interest payments.'}, {'summary_text': 'The corresponding periodic principal and interest payment, labeled as “principal and interest;” (A) (B) (C) (D) (E) (F)'}, {'summary_text': 'If the periodic payment may increase without regard to an interest rate adjustment, the payment that corresponds to the first such increase and the earliest date on which the increase could occur. (B)'}, {'summary_text': 'If an escrow account will be established, an estimate of the amount of taxes and insurance, including any mortgage insurance or any functional equivalent, payable with each periodic payment. (C)'}, {'summary_text': 'The sum of the amounts disclosed underparagraphs (s)(3)(i)(A)and(C) of this section is labeled as “total estimated monthly payment.”'}, {'summary_text': 'If the loan is an interest-only loan, for each interest rate disclosed underparagraph (s)(2)(i)of this section, the corresponding periodic payment and. (ii)Interest-only payments.'}, {'summary_text': 'If the payment will be applied to only accrued interest, the amount applied to interest, labeled as “interest payment,” and a statement that none of the payment is being applied to principal;. (A)'}, {'summary_text': 'If the payment will be applied to accrued interest and principal, an itemization of the amount of the first such payment applied will be given. (B)'}, {'summary_text': 'The escrow information described inparagraph (s)(3)(i)(C)of this section. (C) The escrow info described in paragraph (s) (3) of this section; and.'}, {'summary_text': 'The sum of all amounts required to be disclosed underparagraphs (s)(3)(ii)(A)and(C) of this section. (D) The total estimated monthly payment.'}, {'summary_text': 'Payments for negative amortization loans. (4) Payments for Negative Amortization Loans. (5) Payments for Negative amortizations. (6)'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}, {'summary_text': 'The minimum periodic payment required until the first payment increase or interest rate increase, corresponding to the interest rate disclosed underparagraph (s)(2)(ii)(A)of this section.'}, {'summary_text': 'The minimum periodic payment that would be due at the first payment increase and the second, if any, corresponding to the interest rates described inparagraphs (s)(2)(ii)(C)and(D)of this section.'}, {'summary_text': 'A statement that the minimum payment pays only some interest, does not repay any principal, and will cause the loan amount to increase. (C)'}, {'summary_text': 'The fully amortizing periodic payment amount at the earliest time when such a payment must be made. The interest rate corresponding to the interest rate disclosed underparagraph (s)(2)(ii)(B) of this section.'}, {'summary_text': 'If applicable, for each interest rate disclosed underparagraph (s)(2)(ii)of this section, the amount of the fully amortizing periodic payment, labeled as the “full payment option,” and a statement that these payments pay all principal and all accrued interest.'}, {'summary_text': 'Balloon payments. (5) Balloon payments are a form of payment for the use of balloons in the U.S. government.'}, {'summary_text': 'If the transaction will require a balloon payment, defined as a payment that is more than two times a regular periodic payment, the balloon payment shall be disclosed separately from other periodic payments disclosed in the table.'}, {'summary_text': 'If the balloon payment is scheduled to occur at the same time as another payment required to be disclosed in the table pursuant toparagraph (s)(3)or(s)(4) of this section, then that payment must be disclosed. (ii)'}, {'summary_text': 'For a negative amortization loan, the following information, with headings, content, and format substantially similar to Model Clause H–4(G) in appendix H to this part, is required.'}, {'summary_text': 'The maximum interest rate, the shortest period of time in which such interest rate could be reached, the amount of estimated taxes and insurance included in each payment disclosed, and a statement that the loan offers payment options are shown.'}, {'summary_text': "The dollar amount of the increase in the loan's principal balance if the consumer makes only the minimum required payments. The earliest date on which the consumer must begin making fully amortizing payments, assuming that the maximum interest rate is reached at the earliest possible time."}, {'summary_text': 'For purposes of this§ 1026.18(s), we use the term “council’s” definition. (7) We also use the terms “members’ rights” and “consent’.'}, {'summary_text': 'The term “adjustable-rate mortgage” means a transaction secured by real property or a dwelling for which the annual percentage rate may increase after consummation.'}, {'summary_text': 'The term “step-rate mortgage” means a transaction secured by real property or a dwelling for which the interest rate will change after consummation. The rates that will apply and the periods for which they will apply are known at consummations.'}, {'summary_text': 'The term “fixed-rate mortgage” means a transaction secured by real property or a dwelling that is not an adjustable- rate mortgage or a step-rate Mortgage. (iii)'}, {'summary_text': 'The term “interest-only” means that, under the terms of the legal obligation, one or more of the periodic payments may be applied solely to accrued interest and not to loan principal. “Interest-only loan” is a loan that permits interest-only payments.'}, {'summary_text': 'The term “amortizing loan” means a loan in which payment of the periodic payments does not result in an increase in the principal balance under the terms of the legal obligation. The term ‘negative amortization loan’ means a. minimum periodic payment that covers only a portion of the accrued interest.'}, {'summary_text': 'The term “fully-indexed rate” means the interest rate calculated using the index value and margin at the time of consummation. (vi)'}, {'summary_text': '“No-guarantee-to-refinance’ statement. (t)’s’ ‘No-Guarantee’ and ‘‘’’ are the words used to describe the statement.'}, {'summary_text': 'For a closed-end transaction secured by real property or a dwelling, other than a transaction that is subject to§ 1026.19(e)and(f), the creditor shall disclose that there is no guarantee the consumer can refinance the transaction.'}, {'summary_text': 'The statement required byparagraph (t)(1)of this section must be in a form substantially similar to Model Clause H–4(K) in appendix H to this part. (2)'}, {'summary_text': 'Mortgage transactions subject to RESPA are subject to the RESPA. (a) Mortgage Transactions subject to ResPA— (a), (b) Mortgage Transactions subjectto RESPA—'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Visit CNN.com/Travel each week for a new gallery of snapshots from around the world.'}, {'summary_text': "Creditor must provide the consumer with good faith estimates of the disclosures required by§ 1026.18and shall deliver or place them in the mail not later than the third business day after the creditor receives the consumer's written application."}, {'summary_text': 'Creditors may not impose a fee on a consumer before the consumer has received the disclosures required by paragraph (a)(1)(i) of this section. If the disclosures are mailed to the consumer, the consumer is considered to have received them three business days after they are mailed.'}, {'summary_text': "A creditor or other person may impose a fee for obtaining the consumer's credit history before the consumer has received the disclosures required by this section. The fee isbona fide and reasonable in amount."}, {'summary_text': 'Waiting periods for early disclosures and corrected disclosures. (2)Waiting period for early disclosure and corrected disclosure. (3) Waiting periods for late disclosures and corrective disclosures.'}, {'summary_text': 'The creditor shall deliver or place in the mail the good faith estimates required byparagraph (a)(1)(i)of this section not later than the seventh business day before consummation of the transaction. (i)'}, {'summary_text': 'If the annual percentage rate disclosed underparagraph (a)(1)(i)of this section becomes inaccurate, the creditor shall provide corrected disclosures with all changed terms. The consumer must receive the corrected disclosures no later than three business days before consummation.'}, {'summary_text': "Consumer's waiver of waiting period before consummation. (3) If the consumer determines that the extension of credit is needed to meet abona fidepersonal financial emergency, the consumer may modify or waive the seven-business-day waiting period."}, {'summary_text': 'You are not required to complete this agreement merely because you have received these disclosures or signed a loan application. The disclosure required by this paragraph shall be grouped together with the disclosures required byparagraphs (a)(1)or(a)(2) of this section.'}, {'summary_text': "If the annual percentage rate may increase after consummation in a transaction secured by the consumer's principal dwelling with a term greater than one year, the following disclosures must be provided at the time an application form is provided. (b)Certain variable-rate transactions."}, {'summary_text': 'The booklet titledConsumer Handbook on Adjustable Rate Mortgages is a suitable substitute for this guide. (1) The booklet titled Consumer Handbook onadjustable ratemortgages,or a suitable replacement, is available in the U.S.'}, {'summary_text': 'A loan program disclosure for each variable-rate program in which the consumer expresses an interest. (2) The following disclosures, as applicable, shall be provided:'}, {'summary_text': 'The fact that the interest rate, payment, or term of the loan can change. (i) The amount of money owed to you. (ii) The length of the debt.'}, {'summary_text': 'The index or formula used in making adjustments, and a source of information about the index. (ii) The index or formulas used to make adjustments.'}, {'summary_text': 'An explanation of how the interest rate and payment will be determined. (ii) An explanation how the index is adjusted, such as by the addition of a margin. (iii)'}, {'summary_text': 'A statement that the consumer should ask about the current margin value and current interest rate. (iv) A statement that a consumer should Ask About the Current Margin Value and Current Interest Rate. (v)'}, {'summary_text': 'The fact that the interest rate will be discounted, and a statement that the consumer should ask about the amount of theinterest rate discount. (v)'}, {'summary_text': 'The frequency of interest rate and payment changes. (vi) The amount of money owed to the bank. The rate of interest and the amount owed. The interest rate on the money owed.'}, {'summary_text': 'Any rules relating to changes in the index, interest rate, payment amount, and outstanding loan balance including, for example, an explanation of interest rate or payment limitations, negative amortization, and interest rate carryover.'}, {'summary_text': 'At the option of the creditor, either of the following:. (vii) or (vIII) or both of the options above.'}, {'summary_text': 'A historical example, based on a $10,000 loan amount, illustrates how payments and the loan balance would have been affected by interest rate changes. The example shall reflect the most recent 15 years of index values.'}, {'summary_text': 'The maximum interest rate and payment for a $10,000 loan originated at the initial interest rate (index value plus margin) in effect as of an identified month and year.'}, {'summary_text': 'An explanation of how the consumer may calculate the payments for the loan amount to be borrowed based on either:. (ix) or. (x)'}, {'summary_text': 'The most recent payment shown in the historical example inparagraph (b)(2)(viii)(A)of this section; or. (A)'}, {'summary_text': 'The initial interest rate used to calculate the maximum interest rate and payment inparagraph (b)(2)(viii)(B)of this section. (B)'}, {'summary_text': 'The fact that the loan program contains a demand feature. (x) The fact that there is a demand for loans. ( x) The amount of money that can be loaned.'}, {'summary_text': 'The type of information that will be provided in notices of adjustments and the timing of such notices. (xi) The type and timing of notices.'}, {'summary_text': "A statement that disclosure forms are available for the creditor's other variable-rate loan programs. (xii) A statement that the creditor is available to provide disclosure forms for its other variable rate loan programs, as well."}, {'summary_text': 'For an application that is accessed by the consumer in electronic form, the disclosures required byparagraph (b)of this section may be provided to the consumer. (c)Electronic disclosures.'}, {'summary_text': 'Information provided in accordance with variable-rate regulations of other Federal agencies may be substituted for the disclosures required by paragraph (b) of this section.'}, {'summary_text': 'Mortgage loans—early disclosures— are an important part of the U.S. mortgage market. The federal government has been cracking down on mortgage fraud since the financial crisis.'}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York City for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}, {'summary_text': 'In a closed-end consumer credit transaction secured by real property or a cooperative unit, other than a reverse mortgage, the creditor shall provide the consumer with good faith estimates of the disclosures in§ 1026.37.'}, {'summary_text': 'Mortgage broker. (i) Mortgage lender. (ii) Mortgage agent. (I) Mortgage broker.(ii) mortgage broker.'}, {'summary_text': "If a mortgage broker receives a consumer's application, either the creditor or the mortgage broker shall provide a consumer with the disclosures required underparagraph (e)(1)(i)of this section. The creditor shall ensure that such disclosures are provided in accordance with all requirements of this paragraph."}, {'summary_text': 'If a mortgage broker provides any disclosure under§ 1026.19(e), the mortgage broker shall also comply with the requirements of\xa01026.25(c).'}, {'summary_text': 'Timing. (i) Time. (ii) Time to act. (iii) timing. (iv) Time for action. (v) Timing.'}, {'summary_text': "The creditor shall deliver or place in the mail the disclosures required underparagraph (e)(1)(i) of this section not later than the third business day after the creditor receives the consumer's application."}, {'summary_text': 'The creditor shall deliver or place in the mail the disclosures required underparagraph (e)(1)(i)of this section not later than the seventh business day before consummation of the transaction.'}, {'summary_text': "For a transaction secured by a consumer's interest in a timeshare plan described in11 U.S.C. 101(53D),paragraph (e)(1)(iii)(B)of this section does not apply."}, {'summary_text': 'If any disclosures required underparagraph (e)(1)(i)of this section are not provided to the consumer in person, the consumer is considered to have received the disclosures three business days after they are delivered or placed in the mail. (iv)Receipt of early disclosures.'}, {'summary_text': 'The consumer may modify or waive the seven-business-day waiting period for early disclosures required underparagraph (e)(1)(iii)(B) of this section. The consumer must give the creditor a dated written statement that describes the emergency.'}, {'summary_text': 'Shopping for settlement service providers. (vi) Shoppers can choose from a range of service providers to settle their claims. Shoppers may also choose to opt out of settlement altogether.'}, {'summary_text': 'A creditor permits a consumer to shop for a settlement service if the creditor permits the consumer to select the provider of that service. (A)'}, {'summary_text': 'The creditor shall identify the settlement services for which the consumer is permitted to shop in the disclosures required underparagraph (e)(1)(i)of this section. (B)Disclosure of services.'}, {'summary_text': 'The creditor must identify at least one available provider for each settlement service for which the consumer is permitted to shop. The creditor shall provide this written list of settlement service providers separately from the disclosures required byparagraph (e)(1)(i)of this section.'}, {'summary_text': 'Predisclosure activity. (2) The U.S. government has been involved in a number of high-profile cases. (1) A U.N. agency was involved in the death of a U.K. official.'}, {'summary_text': 'Imposition of fees on consumer—. (i)Imposition on consumer by government;. (ii) Imposing fees on consumers by government.'}, {'summary_text': "Fee restriction. Neither a creditor nor any other person may impose a fee on a consumer in connection with the consumer's application for a mortgage transaction. Consumer may indicate an intent to proceed with a transaction in any manner the consumer chooses."}, {'summary_text': "A creditor or other person may impose a bona fide and reasonable fee for obtaining the consumer's credit report before the consumer has received the disclosures required underparagraph (e)(1)(i) of this section. (B)Exception to fee restriction."}, {'summary_text': 'If a creditor or other person provides a consumer with a written estimate of terms or costs specific to that consumer before the consumer receives the disclosures required underparagraph (e)(1)(i) of this section, the creditor or such person shall clearly and conspicuously state at the top of the front of the first page of the estimate in a font size that is no smaller than 12-point font. The written estimate may not be made with headings, content, and format substantially similar to form H–24 or H–25 of appendix H to this part.'}, {'summary_text': "The creditor or other person shall not require a consumer to submit documents verifying information related to the consumer's application before providing the disclosures required by this section. (iii)Verification of information."}, {'summary_text': 'Good faith determination for estimates of closing costs. (3) Good faith determination that the project will be completed on time. (4) A good faith determination of the cost of the project to taxpayers.'}, {'summary_text': 'An estimated closing cost disclosed pursuant toparagraph (e)of this section is in good faith if the charge paid by or imposed on the consumer does not exceed the amount originally disclosed. (i) General rule.'}, {'summary_text': 'An estimate of a charge for a third-party service or a recording fee is in good faith if:. (ii)Limited increases permitted for certain charges.'}, {'summary_text': 'The aggregate amount of charges for third-party services and recording fees paid by or imposed on the consumer does not exceed the aggregate amount disclosed underparagraph (e)(1)(i) of this section by more than 10 percent.'}, {'summary_text': 'The charge for the third-party service is not paid to the creditor or an affiliate of the creditor; and. (B) The charge is paid to a third party, not the creditor.'}, {'summary_text': 'The creditor permits the consumer to shop for the third-party service, consistent withparagraph (e)(1)(vi) of this section. (C)'}, {'summary_text': 'An estimate of any of the charges specified in thisparagraph (e)(3)(iii)is in good faith if it is consistent with the best information reasonably available to the creditor at the time it is disclosed. (iii) Variations permitted for certain charges.'}, {'summary_text': 'Prepaid interest;. (A) pre-paid interest, (B) prepaid interest and (C) paid interest. (D) pre paid interest and paid interest; (E) prepaid interest.'}, {'summary_text': 'Property insurance premiums;. (B) Property insurance premiums, and. (C) property insurance premiums. (D) Property Insurance premiums; and.'}, {'summary_text': 'Amounts placed into an escrow, impound, reserve, or similar account;. (C) Amounts placed in escrow or impound;.'}, {'summary_text': 'Charges paid to third-party service providers selected by the consumer consistent withparagraph (e)(1)(vi)(A)of this section that are not on the list provided under paragraph (e) of this section. (D)'}, {'summary_text': 'Property taxes and other charges paid for third-party services not required by the creditor. (E) Creditor pays for services that are not required.'}, {'summary_text': 'Revised estimates. A creditor may use a revised estimate of a charge instead of the estimate of the charge originally disclosed underparagraph (e)(1)(i)of this section if the revision is due to any of the following reasons:'}, {'summary_text': 'Changed circumstances cause the estimated charges to increase or, in the case of estimated charges identified inparagraph (e)(3)(ii)of this section, cause the aggregate amount to increase by more than 10 percent. For purposes of this paragraph, “changed circumstance” means:'}, {'summary_text': "An extraordinary event beyond the control of any interested party. (1) An unexpected event specific to the consumer or transaction. (2) An event that is outside of the consumer's or transaction's control."}, {'summary_text': 'Information specific to the consumer or transaction that the creditor relied upon when providing the disclosures required underparagraph (e)(1)(i) was inaccurate or changed after the disclosures were provided; or.'}, {'summary_text': 'New information specific to the consumer or transaction that the creditor did not rely on when providing the original disclosures required underparagraph (e)(1)(i)of this section. (3)'}, {'summary_text': "The consumer is ineligible for an estimated charge previously disclosed because a changed circumstance, as defined underparagraph (e)(3)(iv)(A)of this section, affected the consumer's creditworthiness or the value of the security for the loan. (B)Changed circumstance affecting eligibility."}, {'summary_text': 'The consumer requests revisions to the credit terms or the settlement that cause an estimated charge to increase. (C)Revisions requested by the consumer.'}, {'summary_text': 'The points or lender credits change because the interest rate was not locked when the disclosures required underparagraph (e)(1)(i)of this section were provided. No later than three business days after the date the interest rates is locked, the creditor shall provide a revised version of the disclosures.'}, {'summary_text': 'The consumer indicates an intent to proceed with the transaction more than 10 business days, or more than any additional number of days specified by the creditor before the offer expires. (E)Expiration.'}, {'summary_text': 'In transactions involving new construction, where the creditor reasonably expects that settlement will occur more than 60 days after the disclosures required underparagraph (e)(1)(i)of this section, the creditor may provide revised disclosures to the consumer. (F)Delayed settlement date on a construction loan.'}, {'summary_text': 'Provision and receipt of revised disclosures. (4)Provision, receipt and revised disclosures for the year ending December 31, 2013. (5)'}, {'summary_text': 'General rule. If a creditor uses a revised estimate pursuant toparagraph (e)(3)(iv)of this section for the purpose of determining good faith, the creditor must provide a revised version of the disclosures required.'}, {'summary_text': 'The consumer must receive any revised version of the disclosures required underparagraph (e)(1)(i)of this section not later than four business days prior to consummation. If the revised version is not provided to the consumer in person, the consumer is considered to have received such version three business days after the creditor delivers or places such version in the mail.'}, {'summary_text': "Mortgage loans—final disclosures— are a key part of the federal government's financial reporting process. (f) The federal government is required to report on the progress of its mortgage portfolio."}, {'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York City for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}, {'summary_text': 'In a transaction subject to this section, the creditor shall provide the consumer with the disclosures required under§ 1026.38reflecting the actual terms of the transaction. (i)Scope.'}, {'summary_text': 'Timing and timing are key to success. (i) Time is key. (ii) Timing is important. (iii) timing is crucial. (iv) timing can be delayed.'}, {'summary_text': 'Creditor shall ensure that the consumer receives the disclosures required underparagraph (f)(1)(i)of this section no later than three business days before consummation.'}, {'summary_text': "For transactions secured by a consumer's interest in a timeshare plan described in11 U.S.C. 101(53D), the creditor shall ensure that the consumer receives the disclosures required underparagraph (f)(1)(i)of this section no later than consummation."}, {'summary_text': 'If any disclosures required underparagraph (f)(1)(i)of this section are not provided to the consumer in person, the consumer is considered to have received the disclosures three business days after they are delivered or placed in the mail. (iii)Receipt of disclosures.'}, {'summary_text': "Consumer's waiver of waiting period before consummation. If the consumer determines that the extension of credit is needed to meet a bona fide personal financial emergency, the consumer may modify or waive the three-business-day waiting period. The consumer must give the creditor a dated written statement that describes the emergency."}, {'summary_text': 'A settlement agent may provide a consumer with the disclosures required under paragraph (f)(1)(i) of this section. The creditor shall ensure that such disclosures are provided in accordance with all requirements of thisparagraph (f)'}, {'summary_text': 'Subsequent changes were made to the law. The law was changed to make it easier for people to sue the government. The change was made to make the law easier to enforce.'}, {'summary_text': 'Changes before consummation not requiring a new waiting period. If the disclosures provided underparagraph (f)(1)(i) of this section become inaccurate, the creditor shall provide corrected disclosures reflecting any changed terms.'}, {'summary_text': 'Changes before consummation requiring a new waiting period. (ii)Changes before Consummation requiring an additional period of time. (i) A change in the terms of an agreement that requires the consumer to wait longer to receive a corrected disclosure.'}, {'summary_text': 'The annual percentage rate disclosed under§ 1026.38(o)(4)becomes inaccurate, as defined in\xa01026.22. (A)'}, {'summary_text': 'The loan product is changed, causing the information disclosed under§ 1026.38(a)(5)(iii)to become inaccurate. (B)'}, {'summary_text': 'A prepayment penalty is added, causing the statement regarding a prepayments penalty required under§ 1026.38(b)to become inaccurate. (C)'}, {'summary_text': 'Changes due to events occurring after consummation. If an event in connection with the settlement of the transaction occurs that causes the disclosures required underparagraph (f)(1)(i)of this section to become inaccurate, the creditor shall deliver or place in the mail corrected disclosures not later than 30 days after receiving information sufficient to establish that such event has occurred.'}, {'summary_text': 'Changes due to clerical errors. A creditor does not violateparagraph (f)(1)(i) of this section if the disclosures provided under paragraph (f)? (iv) If the creditor delivers or places in the mail corrected disclosures no later than 60 days after consummation.'}, {'summary_text': 'If amounts paid by the consumer exceed the amounts specified underparagraph (e)(3)(i)or(ii)of this section, the creditor refunds the excess to the consumer no later than 60 days after consummation. (v)Refunds related to the good faith analysis.'}, {'summary_text': 'Charges disclosed. (3)Charges disclosed—Charges disclosed; no charges have been filed. (2) Charges disclosed; charges have not been filed; charges are pending.'}, {'summary_text': 'The amount imposed upon the consumer for any settlement service shall not exceed the amount actually received by the settlement service provider for that service. (i)Actual charge.'}, {'summary_text': 'A creditor or settlement service provider may charge a consumer or seller the average charge for a settlement service if the following conditions are satisfied. (ii)Average charge.'}, {'summary_text': 'The average charge is no more than the average amount paid for that service by or on behalf of all consumers and sellers for a class of transactions. (A)'}, {'summary_text': 'The creditor or settlement service provider defines the class of transactions based on an appropriate period of time, geographic area, and type of loan. (B)'}, {'summary_text': 'The creditor or settlement service provider uses the same average charge for every transaction within the defined class; and. (C) The creditor or Settlement service provider using the sameAverage Charge for Every Transaction within thedefined class.'}, {'summary_text': 'The creditor or settlement service provider does not use an average charge:. (D) The creditor or Settlement Service Provider does not Use an Average Charge:.'}, {'summary_text': 'For any type of insurance;. (1) For any kind of insurance. (2) For insurance of any kind. (3) For life insurance; (4) For health insurance.'}, {'summary_text': 'For any charge based on the loan amount or property value; or. (2) For any charge that is based on a loan amount. or.'}, {'summary_text': 'If doing so is otherwise prohibited by law. (3) If doing so, it is illegal to do so. (4) If it is against the law, then it is not allowed.'}, {'summary_text': 'Transactions involving a seller can be traced back to the U.S. Postal Service. The agency tracks transactions involving people who buy and sell items. It also tracks transactions that involve people who sell goods.'}, {'summary_text': "In a transaction subject toparagraph (e)(1)(i)of this section that involves a seller, the settlement agent shall provide the seller with the disclosures in§ 1026.38that relate to the seller's transaction."}, {'summary_text': 'The settlement agent shall provide the disclosures required underparagraph (f)(4)(i)of this section no later than the day of consummation. If an event in connection with the settlement of the transaction occurs that causes disclosures to become inaccurate, such inaccuracy results in a change to the amount actually paid by the seller.'}, {'summary_text': 'The amount imposed on the seller for any settlement service shall not exceed the amount actually received by the service provider for that service, except as otherwise provided inparagraph (f)(3)(ii)of this section. (iii)Charges disclosed.'}, {'summary_text': "When the consumer's and seller's disclosures under thisparagraph (f)are provided on separate documents, as permitted under§ 1026.38(t)(5), the settlement agent shall provide to the creditor a copy of the disclosures provided to the seller. (iv)"}, {'summary_text': 'No fee may be imposed on any person, as a part of settlement costs or otherwise, by a creditor or by a servicer. (5)'}, {'summary_text': 'Special information booklet at time of application. (g) Special information booklet for parents of children with disabilities. (h) For confidential support, call the Samaritans on 08457 90 90 90, visit a local Samaritans branch or see www.samaritans.org.'}, {'summary_text': 'Creditor to provide special information booklet. Creditor must provide copy of booklet to a consumer applying for a consumer credit transaction secured by real property or a cooperative unit.'}, {'summary_text': "The creditor shall deliver or place in the mail the special information booklet not later than three business days after the consumer's application is received. If a consumer uses a mortgage broker, the mortgage broker shall provide the special info booklet and the creditor need not do so."}, {'summary_text': 'In the case of a home equity line of credit subject to§ 1026.40, a creditor or mortgage broker that provides the consumer with a copy of the brochure entitled “When Your Home is On the Line: What You Should Know About Home Equity Lines of Credit,” is deemed to be in compliance.'}, {'summary_text': 'Creditor or mortgage broker need not provide the booklet to the consumer for a transaction that is not the purchase of a one-to-four family residential property.'}, {'summary_text': 'Refinancing transactions;. (A)Refinanced transactions; (B) refinancing transactions. (C) Refinancing deals; (D) refinanced transactions.'}, {'summary_text': 'Closed-end loans secured by a subordinate lien; and. (B) Closed- end loans secured with a subordinate Lien.'}, {'summary_text': 'Reverse mortgages. (C)Revertelevision.com. Back to Mail Online home. back to the page you came from.'}, {'summary_text': 'Creditors may not make changes to, deletions from, or additions to the special information booklet other than the changes specified inparagraphs (g)(2)(i)through(iv) of this section. (2)Permissible changes.'}, {'summary_text': "In the “Complaints” section of the booklet, “the Bureau of Consumer Financial Protection” may be substituted for “HUD's Office of RESPA” and “ the RESPA office.”"}, {'summary_text': 'In the “Avoiding Foreclosure” section of the booklet, it is permissible to inform homeowners that they may find information on and assistance in avoiding foreclosures. The reference to the HUD Web site,http://www.hud.gov/foreclosure/, shall not be deleted.'}, {'summary_text': "“The Bureau of Consumer Financial Protection” may be substituted for the reference to the “Board of Governors of the Federal Reserve System.” The contact information for HUD's Office of RESPA and Interstate Land Sales may be removed from the ‘Contact Information’ section of the appendix to the booklet."}, {'summary_text': 'The cover of the booklet may be in any form and may contain any drawings, pictures or artwork, provided that the title appearing on the cover shall not be changed. Names, addresses, and telephone numbers of the creditor or others and similar information may appear on thecover, but no discussion of the matters covered in the booklet shall appear. References to HUD may be changed to references to the Bureau.'}]

